### N-gram language models or how to write scientific papers (4 pts)

We shall train our language model on a corpora of [ArXiv](http://arxiv.org/) articles and see if we can generate a new one!

![img](https://media.npr.org/assets/img/2013/12/10/istock-18586699-monkey-computer_brick-16e5064d3378a14e0e4c2da08857efe03c04695e-s800-c85.jpg)

_data by neelshah18 from [here](https://www.kaggle.com/neelshah18/arxivdataset/)_

_Disclaimer: this has nothing to do with actual science. But it's fun, so who cares?!_

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
# Alternative manual download link: https://yadi.sk/d/_nGyU2IajjR9-w
!wget "https://www.dropbox.com/s/99az9n1b57qkd9j/arxivData.json.tar.gz?dl=1" -O arxivData.json.tar.gz
!tar -xvzf arxivData.json.tar.gz
data = pd.read_json("./arxivData.json")
data.sample(n=5)

--2021-10-07 04:17:38--  https://www.dropbox.com/s/99az9n1b57qkd9j/arxivData.json.tar.gz?dl=1
Resolving www.dropbox.com (www.dropbox.com)... 2620:100:6026:18::a27d:4612
Connecting to www.dropbox.com (www.dropbox.com)|2620:100:6026:18::a27d:4612|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/dl/99az9n1b57qkd9j/arxivData.json.tar.gz [following]
--2021-10-07 04:17:39--  https://www.dropbox.com/s/dl/99az9n1b57qkd9j/arxivData.json.tar.gz
Reusing existing connection to [www.dropbox.com]:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc7848fae0c40dfd74f82cd10176.dl.dropboxusercontent.com/cd/0/get/BXgElUF0v3b0qCm-Kfcp7UiF5_a87tF9Y4vWfrbl4idn813Yvgd1-VDLtHlmsjNP7HSiKC-gfNO90-C-pMG4HS6XupcC-a2t4Y55f2hcaOdSxvj5BuGyk9JeEm8azDUSlj06-q4CpDjpA20aJONbjQgp/file?dl=1# [following]
--2021-10-07 04:17:39--  https://uc7848fae0c40dfd74f82cd10176.dl.dropboxusercontent.com/cd/0/get/BXgElUF0v3b0qCm-Kfcp7UiF5_a87tF9Y4vWfrbl4idn813Yvgd1-VDLt

,author,day,id,link,month,summary,tag,title,year
37821,"[{'name': 'Carlos Gershenson'}, {'name': 'Dirk...",22,1506.06796v2,"[{'rel': 'related', 'href': 'http://dx.doi.org...",6,The slower is faster (SIF) effect occurs when ...,"[{'term': 'nlin.AO', 'scheme': 'http://arxiv.o...",When slower is faster,2015
6790,"[{'name': 'Giulio Caravagna'}, {'name': 'Danie...",7,1706.02386v1,"[{'rel': 'alternate', 'href': 'http://arxiv.or...",6,Learning the structure of dependencies among m...,"[{'term': 'cs.LG', 'scheme': 'http://arxiv.org...",On learning the structure of Bayesian Networks...,2017
9464,"[{'name': 'Thomas Schoenemann'}, {'name': 'Fre...",18,1102.3830v1,"[{'rel': 'alternate', 'href': 'http://arxiv.or...",2,We present the first method to handle curvatur...,"[{'term': 'cs.CV', 'scheme': 'http://arxiv.org...",A linear framework for region-based image segm...,2011
21172,[{'name': 'Ryan Carey'}],19,1709.06275v1,"[{'rel': 'alternate', 'href': 'http://arxiv.or...",9,A value learning system has incentives to foll...,"[{'term': 'cs.AI', 'scheme': 'http://arxiv.org...",Incorrigibility in the CIRL Framework,2017
31222,"[{'name': 'João Monteiro'}, {'name': 'Zahid Ak...",21,1802.07770v2,"[{'rel': 'alternate', 'href': 'http://arxiv.or...",2,Deep neural networks (DNNs) have shown phenome...,"[{'term': 'cs.CV', 'scheme': 'http://arxiv.org...",Generalizable Adversarial Examples Detection B...,2018


In [3]:
# assemble lines: concatenate title and description
lines = data.apply(lambda row: row['title'] + ' ; ' + row['summary'], axis=1).tolist()

sorted(lines, key=len)[:3]

['Differential Contrastive Divergence ; This paper has been retracted.',
 'What Does Artificial Life Tell Us About Death? ; Short philosophical essay',
 'P=NP ; We claim to resolve the P=?NP problem via a formal argument for P=NP.']

### Tokenization

You know the dril. The data is messy. Go clean the data. Use WordPunctTokenizer or something.


In [4]:
# Task: convert lines (in-place) into strings of space-separated tokens. import & use WordPunctTokenizer
from nltk import tokenize
tokenizer = tokenize.WordPunctTokenizer()
lines = [' '.join(tokenizer.tokenize(l.lower())) for l in lines]

In [5]:
assert sorted(lines, key=len)[0] == \
    'differential contrastive divergence ; this paper has been retracted .'
assert sorted(lines, key=len)[2] == \
    'p = np ; we claim to resolve the p =? np problem via a formal argument for p = np .'

### N-Gram Language Model (1point)

A language model is a probabilistic model that estimates text probability: the joint probability of all tokens $w_t$ in text $X$: $P(X) = P(w_1, \dots, w_T)$.

It can do so by following the chain rule:
$$P(w_1, \dots, w_T) = P(w_1)P(w_2 \mid w_1)\dots P(w_T \mid w_1, \dots, w_{T-1}).$$ 

The problem with such approach is that the final term $P(w_T \mid w_1, \dots, w_{T-1})$ depends on $n-1$ previous words. This probability is impractical to estimate for long texts, e.g. $T = 1000$.

One popular approximation is to assume that next word only depends on a finite amount of previous words:

$$P(w_t \mid w_1, \dots, w_{t - 1}) = P(w_t \mid w_{t - n + 1}, \dots, w_{t - 1})$$

Such model is called __n-gram language model__ where n is a parameter. For example, in 3-gram language model, each word only depends on 2 previous words. 

$$
    P(w_1, \dots, w_n) = \prod_t P(w_t \mid w_{t - n + 1}, \dots, w_{t - 1}).
$$

You can also sometimes see such approximation under the name of _n-th order markov assumption_.

The first stage to building such a model is counting all word occurences given N-1 previous words

In [6]:
from tqdm import tqdm
from collections import defaultdict, Counter

# special tokens: 
# - unk represents absent tokens, 
# - eos is a special token after the end of sequence

UNK, EOS = "_UNK_", "_EOS_"

def count_ngrams(lines, n):
    """
    Count how many times each word occured after (n - 1) previous words
    :param lines: an iterable of strings with space-separated tokens
    :returns: a dictionary { tuple(prefix_tokens): {next_token_1: count_1, next_token_2: count_2}}

    When building counts, please consider the following two edge cases
    - if prefix is shorter than (n - 1) tokens, it should be padded with UNK. For n=3,
      empty prefix: "" -> (UNK, UNK)
      short prefix: "the" -> (UNK, the)
      long prefix: "the new approach" -> (new, approach)
    - you should add a special token, EOS, at the end of each sequence
      "... with deep neural networks ." -> (..., with, deep, neural, networks, ., EOS)
      count the probability of this token just like all others.
    """
    counts = defaultdict(Counter)
    # counts[(word1, word2)][word3] = how many times word3 occured after (word1, word2)

    for l in tqdm(lines):
        tokens = [UNK for _ in range(n - 1)] + l.split() + [EOS]
        for ngram in zip(*[tokens[i:] for i in range(n)]):
            counts[ngram[:-1]][ngram[-1]] += 1
    
    return counts


In [7]:
# let's test it
dummy_lines = sorted(lines, key=len)[:100]
dummy_counts = count_ngrams(dummy_lines, n=3)
assert set(map(len, dummy_counts.keys())) == {2}, "please only count {n-1}-grams"
assert len(dummy_counts[('_UNK_', '_UNK_')]) == 78
assert dummy_counts['_UNK_', 'a']['note'] == 3
assert dummy_counts['p', '=']['np'] == 2
assert dummy_counts['author', '.']['_EOS_'] == 1

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [00:00<00:00, 10586.60it/s]


Once we can count N-grams, we can build a probabilistic language model.
The simplest way to compute probabilities is in proporiton to counts:

$$ P(w_t | prefix) = { Count(prefix, w_t) \over \sum_{\hat w} Count(prefix, \hat w) } $$

In [8]:
class NGramLanguageModel:    
    def __init__(self, lines, n):
        """ 
        Train a simple count-based language model: 
        compute probabilities P(w_t | prefix) given ngram counts
        
        :param n: computes probability of next token given (n - 1) previous words
        :param lines: an iterable of strings with space-separated tokens
        """
        assert n >= 1
        self.n = n
    
        counts = count_ngrams(lines, self.n)
        
        # compute token proabilities given counts
        self.probs = defaultdict(Counter)
        
        # populate self.probs with actual probabilities
        for key, counts in tqdm(counts.items()):
            denom = sum(counts.values())
            for last, last_count in counts.items():
                self.probs[key][last] = last_count / denom
            
    def get_possible_next_tokens(self, prefix):
        """
        :param prefix: string with space-separated prefix tokens
        :returns: a dictionary {token : it's probability} for all tokens with positive probabilities
        """
        prefix = prefix.split()
        prefix = prefix[max(0, len(prefix) - self.n + 1):]
        prefix = [ UNK ] * (self.n - 1 - len(prefix)) + prefix
        return self.probs[tuple(prefix)]
    
    def get_next_token_prob(self, prefix, next_token):
        """
        :param prefix: string with space-separated prefix tokens
        :param next_token: the next token to predict probability for
        :returns: P(next_token|prefix) a single number, 0 <= P <= 1
        """
        return self.get_possible_next_tokens(prefix).get(next_token, 0)

Let's test it!

In [9]:
dummy_lm = NGramLanguageModel(dummy_lines, n=3)

p_initial = dummy_lm.get_possible_next_tokens('') # '' -> ['_UNK_', '_UNK_']
assert np.allclose(p_initial['learning'], 0.02)
assert np.allclose(p_initial['a'], 0.13)
assert np.allclose(p_initial.get('meow', 0), 0)
assert np.allclose(sum(p_initial.values()), 1)

p_a = dummy_lm.get_possible_next_tokens('a') # '' -> ['_UNK_', 'a']
assert np.allclose(p_a['machine'], 0.15384615)
assert np.allclose(p_a['note'], 0.23076923)
assert np.allclose(p_a.get('the', 0), 0)
assert np.allclose(sum(p_a.values()), 1)

assert np.allclose(dummy_lm.get_possible_next_tokens('a note')['on'], 1)
assert dummy_lm.get_possible_next_tokens('a machine') == \
    dummy_lm.get_possible_next_tokens("there have always been ghosts in a machine"), \
    "your 3-gram model should only depend on 2 previous words"

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2086/2086 [00:00<00:00, 99808.56it/s]


Now that you've got a working n-gram language model, let's see what sequences it can generate. But first, let's train it on the whole dataset.

In [10]:
lm = NGramLanguageModel(lines, n=3)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1219478/1219478 [00:07<00:00, 153160.00it/s]


The process of generating sequences is... well, it's sequential. You maintain a list of tokens and iteratively add next token by sampling with probabilities.

$ X = [] $

__forever:__
* $w_{next} \sim P(w_{next} | X)$
* $X = concat(X, w_{next})$


Instead of sampling with probabilities, one can also try always taking most likely token, sampling among top-K most likely tokens or sampling with temperature. In the latter case (temperature), one samples from

$$w_{next} \sim {P(w_{next} | X) ^ {1 / \tau} \over \sum_{\hat w} P(\hat w | X) ^ {1 / \tau}}$$

Where $\tau > 0$ is model temperature. If $\tau << 1$, more likely tokens will be sampled with even higher probability while less likely tokens will vanish.

In [11]:
def get_next_token(lm, prefix, temperature=1.0):
    """
    return next token after prefix;
    :param temperature: samples proportionally to lm probabilities ^ (1 / temperature)
        if temperature == 0, always takes most likely token. Break ties arbitrarily.
    """
    tokens, probs = zip(*lm.get_possible_next_tokens(prefix).items())
    if temperature == 0:
        return tokens[np.argmax(probs)]

    heated_probs = np.power(np.array(probs), 1 / (temperature))
    heated_probs /= heated_probs.sum()
    return np.random.choice(tokens, p=heated_probs)

In [12]:
from collections import Counter
test_freqs = Counter([get_next_token(lm, 'there have') for _ in range(10000)])
assert 250 < test_freqs['not'] < 450
assert 8500 < test_freqs['been'] < 9500
assert 1 < test_freqs['lately'] < 200

test_freqs = Counter([get_next_token(lm, 'deep', temperature=1.0) for _ in range(10000)])
assert 1500 < test_freqs['learning'] < 3000
test_freqs = Counter([get_next_token(lm, 'deep', temperature=0.5) for _ in range(10000)])
assert 8000 < test_freqs['learning'] < 9000
test_freqs = Counter([get_next_token(lm, 'deep', temperature=0.0) for _ in range(10000)])
assert test_freqs['learning'] == 10000

print("Looks nice!")

Looks nice!


Let's have fun with this model

In [13]:
prefix = 'artificial' # <- your ideas :)

for i in range(100):
    prefix += ' ' + get_next_token(lm, prefix)
    if prefix.endswith(EOS) or len(lm.get_possible_next_tokens(prefix)) == 0:
        break
        
print(prefix)

artificial neural networks ( cnns ) are publicly available psg records with 57 % over the network was trained on small datasets from real samples to a significant challenge is arising now due to the large scale graphical lasso ( sgl ), for quantifying the causal network inference ; we propose a variant of the primary advantages of the portal , which can be expressed by inconsistency management , creating a demand - i dataset . experimental results on random shuffling of syntactical domains . _EOS_


In [14]:
prefix = 'bridging the' # <- more of your ideas

for i in range(100):
    prefix += ' ' + get_next_token(lm, prefix, temperature=0.5)
    if prefix.endswith(EOS) or len(lm.get_possible_next_tokens(prefix)) == 0:
        break
        
print(prefix)

bridging the gap between the different choices of the proposed method is able to generate an initial state of the proposed approach is based on the data . _EOS_


__More in the homework:__ nucleous sampling, top-k sampling, beam search(not for the faint of heart).

### Evaluating language models: perplexity (1point)

Perplexity is a measure of how well does your model approximate true probability distribution behind data. __Smaller perplexity = better model__.

To compute perplexity on one sentence, use:
$$
    {\mathbb{P}}(w_1 \dots w_N) = P(w_1, \dots, w_N)^{-\frac1N} = \left( \prod_t P(w_t \mid w_{t - n}, \dots, w_{t - 1})\right)^{-\frac1N},
$$


On the corpora level, perplexity is a product of probabilities of all tokens in all sentences to the power of 1, divided by __total length of all sentences__ in corpora.

This number can quickly get too small for float32/float64 precision, so we recommend you to first compute log-perplexity (from log-probabilities) and then take the exponent.

In [15]:
def perplexity(lm, lines, min_logprob=np.log(10 ** -50.)):
    """
    :param lines: a list of strings with space-separated tokens
    :param min_logprob: if log(P(w | ...)) is smaller than min_logprop, set it equal to min_logrob
    :returns: corpora-level perplexity - a single scalar number from the formula above
    
    Note: do not forget to compute P(w_first | empty) and P(eos | full_sequence)
    
    PLEASE USE lm.get_next_token_prob and NOT lm.get_possible_next_tokens
    """
    log_prob, N = 0.0, 0
    print('computing perplexity')
    for l in tqdm(lines):
        raw_tokens = l.split()
        N += len(raw_tokens) + 1
        tokens = [UNK for _ in range(lm.n - 1)] + raw_tokens + [EOS]

        log_prob_local = 0.0
    
        for ngram in zip(*[tokens[i:] for i in range(lm.n)]):
            prob = lm.get_next_token_prob(' '.join(ngram[:-1]), ngram[-1])
            log_prob_local += max(min_logprob, np.log(prob))

        log_prob += log_prob_local

    log_perp = - (1 / N) * log_prob 
    return np.exp(log_perp)

In [16]:
lm1 = NGramLanguageModel(dummy_lines, n=1)
lm3 = NGramLanguageModel(dummy_lines, n=3)
lm10 = NGramLanguageModel(dummy_lines, n=10)

ppx1 = perplexity(lm1, dummy_lines)
ppx3 = perplexity(lm3, dummy_lines)
ppx10 = perplexity(lm10, dummy_lines)
ppx_missing = perplexity(lm3, ['the jabberwock , with eyes of flame , '])  # thanks, L. Carrol

print("Perplexities: ppx1=%.3f ppx3=%.3f ppx10=%.3f" % (ppx1, ppx3, ppx10))

assert all(0 < ppx < 500 for ppx in (ppx1, ppx3, ppx10)), "perplexity should be nonnegative and reasonably small"
assert ppx1 > ppx3 > ppx10, "higher N models should overfit and "
assert np.isfinite(ppx_missing) and ppx_missing > 10 ** 6, "missing words should have large but finite perplexity. " \
    " Make sure you use min_logprob right"
assert np.allclose([ppx1, ppx3, ppx10], (318.2132342216302, 1.5199996213739575, 1.1838145037901249))

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2703/2703 [00:00<00:00, 102946.63it/s]


computing perplexity


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [00:00<00:00, 8026.92it/s]


computing perplexity


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [00:00<00:00, 6732.43it/s]


computing perplexity


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [00:00<00:00, 5066.20it/s]


computing perplexity


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 1286.60it/s]

Perplexities: ppx1=318.213 ppx3=1.520 ppx10=1.184


Now let's measure the actual perplexity: we'll split the data into train and test and score model on test data only.

In [17]:
from sklearn.model_selection import train_test_split
train_lines, test_lines = train_test_split(lines, test_size=0.25, random_state=42)

for n in (1, 2, 3):
    lm = NGramLanguageModel(n=n, lines=train_lines)
    ppx = perplexity(lm, test_lines)
    print("N = %i, Perplexity = %.5f" % (n, ppx))


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 51.32it/s]


computing perplexity


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10250/10250 [00:07<00:00, 1373.01it/s]


N = 1, Perplexity = 1832.23136


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 54176/54176 [00:00<00:00, 64293.56it/s]


...

N = 2, Perplexity = 85653987.28543


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1005464/1005464 [00:06<00:00, 146926.57it/s]


computing perplexity


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10250/10250 [00:14<00:00, 720.17it/s]

N = 3, Perplexity = 61999196239911532363776.00000


In [18]:
# whoops, it just blew up :)

### LM Smoothing

The problem with our simple language model is that whenever it encounters an n-gram it has never seen before, it assigns it with the probabilitiy of 0. Every time this happens, perplexity explodes.

To battle this issue, there's a technique called __smoothing__. The core idea is to modify counts in a way that prevents probabilities from getting too low. The simplest algorithm here is Additive smoothing (aka [Lapace smoothing](https://en.wikipedia.org/wiki/Additive_smoothing)):

$$ P(w_t | prefix) = { Count(prefix, w_t) + \delta \over \sum_{\hat w} (Count(prefix, \hat w) + \delta) } $$

If counts for a given prefix are low, additive smoothing will adjust probabilities to a more uniform distribution. Not that the summation in the denominator goes over _all words in the vocabulary_.

Here's an example code we've implemented for you:

In [19]:
class LaplaceLanguageModel(NGramLanguageModel): 
    """ this code is an example, no need to change anything """
    def __init__(self, lines, n, delta=1.0):
        self.n = n
        counts = count_ngrams(lines, self.n)
        self.vocab = set(token for token_counts in counts.values() for token in token_counts)
        self.probs = defaultdict(Counter)

        for prefix in counts:
            token_counts = counts[prefix]
            total_count = sum(token_counts.values()) + delta * len(self.vocab)
            self.probs[prefix] = {token: (token_counts[token] + delta) / total_count
                                          for token in token_counts}
    def get_possible_next_tokens(self, prefix):
        token_probs = super().get_possible_next_tokens(prefix)
        missing_prob_total = 1.0 - sum(token_probs.values())
        missing_prob = missing_prob_total / max(1, len(self.vocab) - len(token_probs))
        return {token: token_probs.get(token, missing_prob) for token in self.vocab}
    
    def get_next_token_prob(self, prefix, next_token):
        token_probs = super().get_possible_next_tokens(prefix)
        if next_token in token_probs:
            return token_probs[next_token]
        else:
            missing_prob_total = 1.0 - sum(token_probs.values())
            missing_prob_total = max(0, missing_prob_total) # prevent rounding errors
            return missing_prob_total / max(1, len(self.vocab) - len(token_probs))
        

In [20]:
#test that it's a valid probability model
for n in (1, 2, 3):
    dummy_lm = LaplaceLanguageModel(dummy_lines, n=n)
    assert np.allclose(sum([dummy_lm.get_next_token_prob('a', w_i) for w_i in dummy_lm.vocab]), 1), "I told you not to break anything! :)"

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [00:00<00:00, 14062.58it/s]


In [21]:
for n in (1, 2, 3):
    lm = LaplaceLanguageModel(train_lines, n=n, delta=0.1)
    ppx = perplexity(lm, test_lines)
    print("N = %i, Perplexity = %.5f" % (n, ppx))

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 30750/30750 [00:03<00:00, 10234.00it/s]


computing perplexity


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10250/10250 [00:14<00:00, 718.83it/s]


N = 1, Perplexity = 977.67559


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 30750/30750 [00:05<00:00, 6135.11it/s]


...

N = 2, Perplexity = 470.48021


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 30750/30750 [00:10<00:00, 2863.78it/s]


computing perplexity


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10250/10250 [00:14<00:00, 717.79it/s]

N = 3, Perplexity = 3679.44765


In [22]:
lm = LaplaceLanguageModel(train_lines, n=5, delta=0.1)
prefix = 'artificial' # <- your ideas :)

for i in range(100):
    prefix += ' ' + get_next_token(lm, prefix)
    if prefix.endswith(EOS) or len(lm.get_possible_next_tokens(prefix)) == 0:
        break
        
print(prefix)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 30750/30750 [00:20<00:00, 1476.83it/s]


artificial comming aod airborne numerational bioimaging depm internal bologna abilene bioquery rows grm sparsemax cortex trajecotries holyhammer gleaned sbmt hcrf reflux fiss parental xom textons waned allegro meier oscillated fci lbm macrocolumn loo rsvgd deeplda postedition correcting composing v6 solutions cultivated classificatory milcnn nesterov faceboxes forth fiabilité stdev isbi odors keyboard ofek moore nadl darija nuclearity rahimi07 699 fiji diametric 669 ation motionsense batchppo nmr2000 compas became agree cll chdp psychoanalysis gets polypeptides perceptrons deducted esotery outlying mucosa coreferential 15855m chance nir faber subclassof scatter f_e fopl birthday fashioned bigann istica vaps exposing inapplicability ssid electroencephalographic exteriors precortical interact knockoff indisputably


### Kneser-Ney smoothing (2 points)

Additive smoothing is simple, reasonably good but definitely not a State of The Art algorithm.


Your final task in this notebook is to implement [Kneser-Ney](https://en.wikipedia.org/wiki/Kneser%E2%80%93Ney_smoothing) smoothing.

It can be computed recurrently, for n>1:

$$P_{kn}(w_t | prefix_{n-1}) = { \max(0, Count(prefix_{n-1}, w_t) - \delta) \over \sum_{\hat w} Count(prefix_{n-1}, \hat w)} + \lambda_{prefix_{n-1}} \cdot P_{kn}(w_t | prefix_{n-2})$$

where
- $prefix_{n-1}$ is a tuple of {n-1} previous tokens
- $lambda_{prefix_{n-1}}$ is a normalization constant chosen so that probabilities add up to 1
- Unigram $P_{kn}(w_t | prefix_{n-2})$ corresponds to Kneser Ney smoothing for {N-1}-gram language model.
- Unigram $P_{kn}(w_t)$ is a special case: how likely it is to see x_t in an unfamiliar context

See lecture slides or wiki for more detailed formulae.

__Your task__ is to
- implement KneserNeyLanguageModel
- test it on 1-3 gram language models
- find optimal (within reason) smoothing delta for 3-gram language model with Kneser-Ney smoothing

In [23]:
class KneserNeyLanguageModel(NGramLanguageModel): 
    """ A template for Kneser-Ney language model. Default delta may be suboptimal. """
    def __init__(self, lines, n, delta=1.0):
        self.n = n
        self.delta = delta
        
        self.prefix_counts = {}
        counts = self.counts = count_ngrams(lines, n)
        
        self.vocab = set(token for token_counts in counts.values() for token in token_counts)
        
        for prefix, counter in self.counts.items():
            self.prefix_counts[prefix] = sum(counter.values())
          
        self.probs = {}
        self.prob_norms = {}
        
        if n == 1:
            normalizer = sum(self.counts[prefix].values())
            self.probs[()] = { 
                k: v / normalizer for k, v in self.counts[prefix].items()
            }
            self.prob_norms[()] = sum(self.probs[()].values())
            
            self.prev_model = None            
        else:
            for prefix, counter in counts.items():
                self.probs[prefix] = {
                    k: (max(v, 0) - self.delta) / self.prefix_counts[prefix]
                    for k, v in self.counts[prefix].items()
                }
                
                self.prob_norms[prefix] = sum(self.probs[prefix].values())
            
            self.prev_model = KneserNeyLanguageModel(lines, n - 1, delta)             
            
        
    def get_possible_next_tokens(self, prefix):
        if self.n == 1:
            prefix = ()
            return self.probs[prefix]
        
        prefix = prefix.split()
        prefix = prefix[max(0, len(prefix) - self.n + 1):]
        prefix = [ UNK ] * (self.n - 1 - len(prefix)) + prefix
        prefix = tuple(prefix)
        
        probs = self.probs.get(prefix, {})
        norm = self.prob_norms.get(prefix, 0.0)
            
        adder_prefix = ' '.join(prefix[1:])
        adder = self.prev_model.get_possible_next_tokens(adder_prefix)

        adder_norm = sum(adder.values())
        adder_coef = (1 - norm) / adder_norm
    
        new_probs = {k: probs.get(k, 0) + adder_coef * va for k, va in adder.items()}
        return new_probs
        
    def get_next_token_prob(self, prefix, next_token):
        if self.n == 1:
            return super().get_next_token_prob(prefix, next_token)
        
        prefix = prefix.split()
        prefix = prefix[max(0, len(prefix) - self.n + 1):]
        prefix = [ UNK ] * (self.n - 1 - len(prefix)) + prefix
        prefix = tuple(prefix)
        
        probs = self.probs.get(prefix, {})
        norm = self.prob_norms.get(prefix, 0.0)
            
        adder_prefix = ' '.join(prefix[1:])
        adder_prob = self.prev_model.get_next_token_prob(adder_prefix, next_token)

        adder_coef = 1 - norm
    
        new_prob = probs.get(next_token, 0) + adder_coef * adder_prob
        return new_prob

In [24]:
#test that it's a valid probability model
for n in (1, 2, 3):
    dummy_lm = KneserNeyLanguageModel(dummy_lines, n=n)
    assert np.allclose(sum([dummy_lm.get_next_token_prob('a', w_i) for w_i in dummy_lm.vocab]), 1), "I told you not to break anything! :)"

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [00:00<00:00, 38033.22it/s]


In [30]:
## grid search

min_ppx_args = None

for delta in np.logspace(np.log(0.5), np.log(1.1), 10):
    print(f'experiment with delta = {delta}')
    # as far as i see it, big deltas make model unigram-biased
    for n in (1, 2, 3):
        lm = KneserNeyLanguageModel(train_lines, n=n, delta=delta)
        print(f'finished building {n}-gram model')
        ppx = perplexity(lm, test_lines)
        
        if min_ppx_args is None or ppx < min_ppx_args[0]:
            min_ppx_args = ppx, delta, n
        
        print("N = %i, Perplexity = %.5f" % (n, ppx))
    print('', end='\n\n\n')

experiment with delta = 0.2026995662865173


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 30750/30750 [00:03<00:00, 10169.07it/s]


finished building 1-gram model
computing perplexity


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10250/10250 [00:05<00:00, 1863.29it/s]


N = 1, Perplexity = 1832.23136


...

N = 2, Perplexity = 12502.12615


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 30750/30750 [00:03<00:00, 10233.07it/s]


finished building 3-gram model
computing perplexity


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10250/10250 [00:22<00:00, 460.08it/s]

N = 3, Perplexity = 766840.56716





In [31]:
min_ppx_args

(298.76236082480096, 0.8319503964950192, 3)

### Min PPX: 298.7623608248009
### Delta: 0.8319503964950
### Ngram: 3